# Imports

In [ ]:
# Standard library imports
import glob

# Local library imports
from src import ImageUtils

# Related 3rd party imports
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from skimage import feature
import seaborn
import numpy as np
import pandas as pd

# Loading images

In [ ]:
paths = glob.glob("/home/pavle/Desktop/Image based biometry/Image-based-biometry/ex1/awe/*/*.png")
images = ImageUtils.create_images_array(paths,(1280,720))

In [ ]:
# print(images)

In [ ]:
    # plt.imshow(images[2],"gray")
    # plt.show()

# Converting matrices to 1D vectors

In [ ]:
d1_vectors = []
for image in images:
    d1_vector = ImageUtils.convert_to_1d_vector(image)
    d1_vectors.append(d1_vector)

# mat = np.vstack(d1_vectors)
# mat.shape

# res = np.zeros(mat.shape[0]*mat.shape[0])
# c = 0
# for i in range(mat.shape[0]):
#     for j in range(mat.shape[0]):
#         res[c] = np.sum(mat[i] - mat[j])
#         c += 1


In [ ]:
# counter = 0
# images_comp = []
# for vec in d1_vectors:
#     if counter != len(d1_vectors):
#         for y in range(counter+1, len(d1_vectors)):
#             vec2 = d1_vectors[y]
#             pixel_comparison=[]
#             for x in range(0, len(vec)):
#                 pixel_comparison.append(vec[x]-vec2[x])
#             images_comp.append(sum(pixel_comparison))
#     counter += 1

# images_comp[0]

# LBP, LBP + Histograms, uLBP (from scikit)

In [ ]:
# test = ImageUtils.lbp_fe(images[93],radio=3)
# print(test)
# plt.imshow(test,"gray")
# plt.show()

In [ ]:
# ImageUtils.lbp_histograms(paths[92:93])

In [ ]:
# ulbp = feature.local_binary_pattern(images[93], 3, 3, method="uniform")
# plt.imshow(ulbp,"gray")

Compare all the computed feature vectors between them-
selves (you get a triangular matrix of distances) separately

for each type of feature extractor (pixel-by-pixel, LBP,
uLBP etc.) and compute rank-1 recognition rate. 

You do that by getting the closest vector to the currently observed
one and check whether it corresponds to the same class
(correct) or not (incorrect).

When you go through all the
vectors and sum up predictions you get the percentage of
rank-1 recognition rate

In [ ]:
lbp_vectors_3 = []
lbp_vectors_4 = []
lbp_vectors_5 = []
histograms = []
ulbp_vectors_3_3= []
ulbp_vectors_4_5 = []

for image in images:
    lbp_vectors_3.append(ImageUtils.lbp_fe(image,radio=3).flatten())
    lbp_vectors_4.append(ImageUtils.lbp_fe(image,radio=4).flatten())
    lbp_vectors_5.append(ImageUtils.lbp_fe(image,radio=5).flatten())
    histograms.append(ImageUtils.lbp_histograms(image,(1280,720)).flatten())
    ulbp_vectors_3_3.append(feature.local_binary_pattern(image, 3, 3, method="uniform").flatten())
    ulbp_vectors_4_5.append(feature.local_binary_pattern(image, 4, 5, method="uniform").flatten())

In [ ]:

# RANK-1 : You do
# that by getting the closest vector to the currently observed
# one and check whether it corresponds to the same class
# (correct) or not (incorrect). When you go through all the
# vectors and sum up predictions you get the percentage of
# rank-1 recognition rate.

def rank_1(vectors):
    counter = 0
    images_comp = []
    for vec in vectors:
        if counter != len(vectors):
            for y in range(counter+1, len(vectors)):
                vec2 = vectors[y]
                dist = np.linalg.norm(vec - vec2)
                images_comp.append(dist)
        counter += 1

    arr = np.array(images_comp)     
    tri = np.zeros((1000, 1000))
    tri[np.triu_indices(1000, 1)] = arr
    df = pd.DataFrame(tri)
    df.replace(0, np.nan, inplace=True) # To find the number the image that is the most similar, we convert 0s to Nans as otherwise image 1's most similar image is image 1. 

    y = 0
    n = 0
    for x in df.columns:
        if (x-x%10)/10 == (df[x].argmin()-df[x].argmin()%10)/10:
            y+= 1
            # print(x, df2[x].argmin())
        else:
            n+= 1
    return (y/n)* 100

 

d1_rank1 = rank_1(d1_vectors)
print(f"Classification rate 1D:   {d1_rank1:.2f} %")
del d1_vectors

lbp_vectors_3_rank1 = rank_1(lbp_vectors_3)
print(f"Classification rate LBP (R=3):  {lbp_vectors_3:.2f} %")

lbp_vectors_4_rank1 = rank_1(lbp_vectors_4)
print(f"Classification rate LBP (R=4):  {lbp_vectors_4:.2f} %")

lbp_vectors_5_rank1 = rank_1(lbp_vectors_5)
print(f"Classification rate LBP (R=5):  {lbp_vectors_5:.2f} %")

histograms_rank1 = rank_1(histograms)
print(f"Classification rate histograms: {histograms:.2f} %")

ulbp_vectors_3_3_rank1 = rank_1(ulbp_vectors_3_3)
print(f"Classification rate uLBP (N=3, R=3): {ulbp_vectors_3_3:.2f} %")

ulbp_vectors_4_5_rank1 = rank_1(ulbp_vectors_4_5)
print(f"Classification rate uLBP (N=4, R=5): {ulbp_vectors_4_5:.2f} %")



# Classification rate 1D:   5.60 %
# Classification rate LBP (radio=3):  5.60 %
# Classification rate LBP (radio=4):  6.27 %
# Classification rate LBP (radio=5):  5.26 %
# Classification rate LBP (radio=6):  2.35 %
# Classification rate uLBP (N=3, radio=3): 8.58 %  # N = Number of circularly symmetric neighbour set points (quantization of the angular space).
# Classification rate uLBP (N=4, radio=5): 6.95 %  # N = Number of circularly symmetric neighbour set points (quantization of the angular space).

In [ ]:
# lbp_rank1 = rank_1(lbp_vectors_5)
# print(f"Classification rate LBP:  {lbp_rank1:.2f} %")
# del lbp_vectors_5

# lbp_rank2 = rank_1(lbp_6)
# print(f"Classification rate LBP:  {lbp_rank2:.2f} %")
# del lbp_6

# ulbp_rank1 = rank_1(ulbp_vectors)
# print(f"Classification rate uLBP: {ulbp_rank1:.2f} %")
# del ulbp_vectors

In [ ]:
# print(f"Classification rate 1D:   {d1_rank1:.2f} %")
# print(f"Classification rate LBP:  {lbp_rank1:.2f} %")
# print(f"Classification rate uLBP: {ulbp_rank1:.2f} %")

In [ ]:
# %matplotlib inline
# plt.figure()
# seaborn.heatmap(df.T)
# plt.show()
# plt.savefig('heatmap.jpg')